In [11]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.models import Model, Sequential
from keras.backend import clear_session
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.activations import sigmoid
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from tensorflow import keras

In [2]:
dataset = pickle.load(open("dataset/IMDB/dataset_IMDB.pickle", "rb"))

In [3]:
Y = np.array(list(map(lambda x: 1 if x=="positive" else 0, dataset["sentiment"])))

In [4]:
x_train,x_test,y_train,y_test = train_test_split(dataset["processed_review"],Y, test_size=0.1, shuffle=True)

In [5]:
def creazione_modello_GloVe(filename):
    f = open(filename, encoding="utf8")
    embeding_index = {}
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeding_index[word] = coefs
    f.close()
    return embeding_index

In [6]:
embedding = creazione_modello_GloVe("dataset/glove.6B.50d.txt")

In [7]:
tokenizer = Tokenizer(num_words=142092)
tokenizer.fit_on_texts(x_train)

word_index = tokenizer.word_index

In [8]:
vocab_len = len(word_index)+1
embedding_vector_len = embedding["banan"].shape[0]
embedding_matrix = np.zeros((vocab_len, embedding_vector_len))

for word, index in word_index.items():
    vector = embedding.get(word)
    if vector is not None:
        embedding_matrix[index, :] = vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embedding_vector_len, input_length=200, weights=[embedding_matrix])

In [24]:
GRU_model = Sequential()
GRU_model.add(embedding_layer)
GRU_model.add(GRU(units=32))
GRU_model.add(Dense(32))
GRU_model.add(Dense(32))
GRU_model.add(Dense(32))
GRU_model.add(Dense(2, activation=sigmoid))

GRU_model.compile(loss="binary_crossentropy", optimizer=RMSprop(), metrics=["accuracy"])

GRU_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 50)           5019250   
_________________________________________________________________
gru_2 (GRU)                  (None, 32)                8064      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_9 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_10 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 66        
Total params: 5,030,548
Trainable params: 5,030,548
Non-trainable params: 0
____________________________________________

In [14]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(45000,) (45000,)
(5000,) (5000,)


In [15]:
x_train_index = tokenizer.texts_to_sequences(x_train)
x_train_index = pad_sequences(x_train_index, maxlen=200)

In [16]:
y_train_categorical = keras.utils.to_categorical(y_train, 2)

In [25]:
clear_session()
GRU_model.fit(x_train_index[:35000], y_train_categorical[:35000], epochs=20, batch_size=32, verbose=1, validation_data=(x_train_index[35000:], y_train_categorical[35000:]))

Epoch 1/20
1094/1094 [==============================] - 81s 72ms/step - loss: 0.6421 - accuracy: 0.6144 - val_loss: 0.5869 - val_accuracy: 0.6883
Epoch 2/20
1094/1094 [==============================] - 79s 72ms/step - loss: 0.5038 - accuracy: 0.7616 - val_loss: 0.4918 - val_accuracy: 0.7777
Epoch 3/20
1094/1094 [==============================] - 78s 72ms/step - loss: 0.4301 - accuracy: 0.8149 - val_loss: 0.4443 - val_accuracy: 0.8050
Epoch 4/20
1094/1094 [==============================] - 78s 72ms/step - loss: 0.3845 - accuracy: 0.8397 - val_loss: 0.4207 - val_accuracy: 0.8227
Epoch 5/20
1094/1094 [==============================] - 79s 72ms/step - loss: 0.3496 - accuracy: 0.8575 - val_loss: 0.4153 - val_accuracy: 0.8254
Epoch 6/20
1094/1094 [==============================] - 79s 72ms/step - loss: 0.3168 - accuracy: 0.8731 - val_loss: 0.3957 - val_accuracy: 0.8381
Epoch 7/20
1094/1094 [==============================] - 79s 72ms/step - loss: 0.2899 - accuracy: 0.8852 - val_loss: 0.3996 -

In [26]:
x_test_index = tokenizer.texts_to_sequences(x_test)
x_test_index = pad_sequences(x_test_index, maxlen=200)

In [27]:
y_pred = GRU_model.predict(x_test_index)

In [28]:
print(classification_report(y_test, np.argmax(y_pred, axis=1).astype("float32")))

              precision    recall  f1-score   support

           0       0.84      0.89      0.87      2503
           1       0.89      0.83      0.86      2497

    accuracy                           0.86      5000
   macro avg       0.86      0.86      0.86      5000
weighted avg       0.86      0.86      0.86      5000



In [29]:
GRU_model.save("IMDB_GRU")

INFO:tensorflow:Assets written to: IMDB_GRU/assets
INFO:tensorflow:Assets written to: IMDB_GRU/assets
